## To reproduce Table 4 results (Cora dataset):

We will reproduce the results for the Cora dataset by default, but you can choose any dataset you want by following the instructions below.

1. Open `scripts/run_table4.sh`
2. Modify as needed:
   - `datasets` (default=Cora)
   - `experiment_name` (Name for the experiment, will be used as directory name for logs)
   - Modify the parallel parameter (-j) to adjust the number of concurrent jobs based on your system's capabilities


3. Make the script executable:
   ```
   chmod +x scripts/run_table4.sh
   ```

4. Run the script:
   ```
   bash scripts/run_table4.sh
   ```

Results will be logged to wandb and saved in the `logs/{experiment_name}` directory.

### Note: To reproduce results for other datasets in Table 4:
- Modify the `datasets` array in the script (e.g., datasets=('CiteSeer') for CiteSeer dataset)
- Change the `experiment_name` accordingly (e.g., "Table 4 results CiteSeer")

For example, to reproduce results for CiteSeer:
1. In `scripts/run_table4.sh`, change:
   ```
   datasets=('CiteSeer')
   experiment_name="Table 4 results CiteSeer"
   ```
2. Run the script as described above

Note: We used GCN architecture for all datasets in Table 4, so keep architectures=('gcn')

### Parsing the results

After running the script, the logs of the experiments are stored in the directory `logs/Table 4 results Cora`.
The code below parses these results to compute the average accuracy for each attack method across multiple seeds.
It extracts the accuracy values for different attack methods at specific epochs:
- For 'gradients', 'features', and 'labels', it uses the accuracy at epoch 0.
- For 'output_server' and 'forward_values', it uses the accuracy at the last epoch.

The code also renames some attack methods to match the names used in the paper:
- 'output_server' is renamed to 'Prediction output'
- 'forward_values' is renamed to 'Inter-Reps'

Don't forget to change the experiment name in the code in the next cell to parse the correct logs of the experiment for the new dataset

In [ ]:
import os
import pandas as pd
import numpy as np

def safe_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

def parse_results(experiment_dir):
    results = []
    for subdir in os.listdir(experiment_dir):
        csv_path = os.path.join(experiment_dir, subdir, 'attack_results.csv')
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            
            # Filter for Accuracy metrics only
            accuracy_df = df[df['metric'].str.startswith('Accuracy-')].copy()
            
            # Convert 'value' column to numeric
            accuracy_df['value'] = accuracy_df['value'].apply(safe_float)
            
            # Get results for gradients, features, labels at epoch 0
            epoch0_results = accuracy_df[accuracy_df['epoch'] == 0]
            for attack in ['gradients', 'features', 'labels']:
                accuracy = epoch0_results[epoch0_results['metric'] == f'Accuracy-{attack}']['value'].values
                if len(accuracy) > 0:
                    results.append({'attack': attack, 'accuracy': accuracy[0]})
            
            # Get results for output_server and forward_values at last epoch
            last_epoch = accuracy_df['epoch'].max()
            last_epoch_results = accuracy_df[accuracy_df['epoch'] == last_epoch]
            for attack in ['output_server', 'forward_values']:
                accuracy = last_epoch_results[last_epoch_results['metric'] == f'Accuracy-{attack}']['value'].values
                if len(accuracy) > 0:
                    attack_name = 'Inter-Reps' if attack == 'forward_values' else 'Prediction output'
                    results.append({'attack': attack_name, 'accuracy': accuracy[0]})
    
    return pd.DataFrame(results)

# Specify the path to your experiment directory
experiment_dir = '../logs/Table 4 results Cora'

# Parse results
results_df = parse_results(experiment_dir)

# Compute mean and std
summary = results_df.groupby('attack')['accuracy'].agg(['mean', 'std']).reset_index()

# Format the results
summary['result'] = summary.apply(lambda row: f"{row['mean']*100:.4f} ± {row['std']*100:.4f}", axis=1)

# Define the desired order
order = ['gradients', 'Inter-Reps', 'features', 'labels', 'Prediction output']

# Reorder the rows
summary = summary.set_index('attack').loc[order].reset_index()

# Display the results
print("---------------- Cora ----------------")
print(summary[['attack', 'result']])